In [12]:
from datetime import date
from dataclasses import dataclass
import math

In [13]:
MIN_YEAR = 2023
MAX_YEAR = 2033
# -1 pefixes will be used in arrays as days and months are not zero-indexed.
DAYS_IN_24_HR_MONTH         = [-1, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
DAYS_IN_24_HR_MONTH_LEAP_YR = [-1, 31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
# DAYS_IN_36_HR_MONTH         = [-1, 21, 19, 21, 20, 21, 20, 21, 21, 20, 21, 20, 21]
# DAYS_IN_36_HR_MONTH_LEAP_YR = [-1, 21, 20, 21, 20, 21, 20, 21, 21, 20, 21, 20, 21]
MONTH_ABRVS = [-1, "JA", "FE", "MR", "AR", "MA", "JN", "JL", "AU", "SE", "OC", "NO", "dE"]

In [32]:
get_conversion_definition(2024,
                          DAYS_IN_24_HR_MONTH,
                          DAYS_IN_24_HR_MONTH_LEAP_YR,
                          36)

[-1,
 Month(num_days=21, hours_in_days=[-1, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 24], month_abrv='JA'),
 Month(num_days=20, hours_in_days=[-1, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 12], month_abrv='FE'),
 Month(num_days=21, hours_in_days=[-1, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 24], month_abrv='MR'),
 Month(num_days=20, hours_in_days=[-1, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36], month_abrv='AR'),
 Month(num_days=21, hours_in_days=[-1, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 24], month_abrv='MA'),
 Month(num_days=20, hours_in_days=[-1, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36], month_abrv='JN'),
 Month(num_days=21, hours_in_days=[-1, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 24], month_abrv='JL'),
 Month(num_da

In [14]:
def is_leap_year(year):
    if year % 4 == 0:
        if year % 100 == 0:
            if year % 400 == 0:
                return True
            else:
                return False
        else:
            return True
    else:
        return False

In [15]:
@dataclass
class Month:
    num_days: int
    hours_in_days: list
    month_abrv: str

In [16]:
def get_conversion_definition(year, days_in_months, days_in_monts_leap_yr, conv_hrs_per_day):
    hrs_in_24_hr_month = [-1]
    days_in_24_hr_month = days_in_monts_leap_yr if is_leap_year(year) else days_in_months
    for day in days_in_24_hr_month[1:]:
        hrs_in_24_hr_month.append(day * 24)
    
    year_definition = [-1]
    for month_hrs, month_abrv in zip(hrs_in_24_hr_month[1:], MONTH_ABRVS[1:]):
        num_converted_days = math.floor(month_hrs / conv_hrs_per_day)
        last_day_hrs = month_hrs % conv_hrs_per_day
        hours_in_days = [conv_hrs_per_day for i in range(num_converted_days)]
        if last_day_hrs != 0:
            hours_in_days.append(last_day_hrs)

        year_definition.append(
            Month(num_days=len(hours_in_days),
                  hours_in_days=[-1] + hours_in_days,
                  month_abrv=month_abrv)
        )
        
    return year_definition

In [222]:
class Date24hrs:
    def days_in_month(self, year, month):
        # Return the days in the given month of the given year, accounting for leap years. 
        assert 1 <= month <= 12, month
        if is_leap_year(year):
            return DAYS_IN_24_HR_MONTH_LEAP_YR[month]
        return DAYS_IN_24_HR_MONTH[month]
    
    def check_date(self, year, month, day, hour):
        # Make sure the provided date is valid.
        if not MIN_YEAR <= year <= MAX_YEAR:
            raise ValueError(f'bad! {year=}')
        if not 1 <= month <= 12:
            raise ValueError(f'bad! {month=}')
        if not 1 <= day <= self.days_in_month(year, month):
            raise ValueError(f'bad! {year=} {month=} {day=}')
        if not 0 <= hour <= 23:
            raise ValueError(f'bad! {hour=}')
    
        return year, month, day, hour
        
    def __init__(self, year, month, day, hour):
        self.year, self.month, self.day, self.hour = self.check_date(year, month, day, hour)
        self.day_length = 24
        self.month_abrv = MONTH_ABRVS[self.month]
        
#     def to_36_hr_date(self) -> 'Date36hrs':
#         # Calculate the total number of hours that have passed from the beginning of the current
#         # month until the the current day.
#         hrs_elapsed_to_day = self.day * self.day_length
#         print(f'{hrs_elapsed_to_day=}')
#         day_converted = math.ceil(hrs_elapsed_to_day / 36)
#         print(f'{day_converted=}')
#         # The last day of each 36hr month might not actually have 36hrs in it (a short day).
#         # Check if the day is a short day.
#         conversion = get_conversion_definition(self.year,
#                                                DAYS_IN_24_HR_MONTH,
#                                                DAYS_IN_24_HR_MONTH_LEAP_YR,
#                                                36)
#         print(f'{(day_converted == conversion[self.month].num_days - 1)=}')
#         print(f'{(conversion[self.month].hours_in_days[-1] != 36)=}')
#         print(f'{(hrs_elapsed_to_day % 36 > 0)=}')

# #         short_day = (day_converted == conversion[self.month].num_days - 1) and \
# #                     (conversion[self.month].hours_in_days[-1] != 36) and \
# #                     (hrs_elapsed_to_day % 36 > 0)
# #         if short_day:
# #             day_converted += 1
#         print(f'{day_converted=}')
#         # Total number of hours from the beginning of the current month until the current
#         # _hour_ of the current day.
#         total_hrs_elapsed = ((self.day - 1) * self.day_length) + self.hour
#         print(f'{total_hrs_elapsed=}')
#         hour_converted = (total_hrs_elapsed - (day_converted * 36)) % conversion[self.month].hours_in_days[day_converted]
#         print(f'{hour_converted=}')
#         return Date36hrs(self.year, self.month, day_converted, hour_converted)\

    def to_36_hr_date(self) -> 'Date36hrs':
        # Calculate the total number of hours that have passed from the beginning of the current
        # month until the the current day.
        total_hrs_elapsed = ((self.day - 1) * self.day_length) + self.hour
        day_converted = math.floor(total_hrs_elapsed / 36) + 1
        conversion = get_conversion_definition(self.year,
                                               DAYS_IN_24_HR_MONTH,
                                               DAYS_IN_24_HR_MONTH_LEAP_YR,
                                               36)
        hour_converted = (total_hrs_elapsed - (math.floor(total_hrs_elapsed / 36) * 36)) \
                         % conversion[self.month].hours_in_days[day_converted]
        return Date36hrs(self.year, self.month, day_converted, hour_converted)

In [223]:
class Date36hrs:
    def __init__(self, year, month, day, hour):
        self.year = year
        self.month = month
        self.day = day
        self.hour = hour
        self.month_abrv = MONTH_ABRVS[self.month]

In [224]:
epoch = Date24hrs(2023, 4, 9, 5).to_36_hr_date()
print(f'{epoch.year=}, {epoch.month=} ({epoch.month_abrv}), {epoch.day=}, {epoch.hour=}')

epoch.year=2023, epoch.month=4 (AR), epoch.day=6, epoch.hour=17


In [225]:
epoch = Date24hrs(2023, 4, 29, 12).to_36_hr_date()
print(f'{epoch.year=}, {epoch.month=} ({epoch.month_abrv}), {epoch.day=}, {epoch.hour=}')

epoch.year=2023, epoch.month=4 (AR), epoch.day=20, epoch.hour=0


In [226]:
epoch = Date24hrs(2024, 2, 29, 23).to_36_hr_date()
print(f'{epoch.year=}, {epoch.month=} ({epoch.month_abrv}), {epoch.day=}, {epoch.hour=}')

epoch.year=2024, epoch.month=2 (FE), epoch.day=20, epoch.hour=11


In [227]:
epoch = Date24hrs(2023, 2, 28, 23).to_36_hr_date()
print(f'{epoch.year=}, {epoch.month=} ({epoch.month_abrv}), {epoch.day=}, {epoch.hour=}')

epoch.year=2023, epoch.month=2 (FE), epoch.day=19, epoch.hour=23


In [228]:
epoch = Date24hrs(2023, 2, 28, 12).to_36_hr_date()
print(f'{epoch.year=}, {epoch.month=} ({epoch.month_abrv}), {epoch.day=}, {epoch.hour=}')

epoch.year=2023, epoch.month=2 (FE), epoch.day=19, epoch.hour=12


In [229]:
for day in range(1, 32):
    print(f'2023 - 1 - {day} : 0')
    epoch = Date24hrs(2023, 1, day, 0).to_36_hr_date()
    print(f'{epoch.year=}, {epoch.month=} ({epoch.month_abrv}), {epoch.day=}, {epoch.hour=}')
    print('-----------------')

2023 - 1 - 1 : 0
epoch.year=2023, epoch.month=1 (JA), epoch.day=1, epoch.hour=0
-----------------
2023 - 1 - 2 : 0
epoch.year=2023, epoch.month=1 (JA), epoch.day=1, epoch.hour=24
-----------------
2023 - 1 - 3 : 0
epoch.year=2023, epoch.month=1 (JA), epoch.day=2, epoch.hour=12
-----------------
2023 - 1 - 4 : 0
epoch.year=2023, epoch.month=1 (JA), epoch.day=3, epoch.hour=0
-----------------
2023 - 1 - 5 : 0
epoch.year=2023, epoch.month=1 (JA), epoch.day=3, epoch.hour=24
-----------------
2023 - 1 - 6 : 0
epoch.year=2023, epoch.month=1 (JA), epoch.day=4, epoch.hour=12
-----------------
2023 - 1 - 7 : 0
epoch.year=2023, epoch.month=1 (JA), epoch.day=5, epoch.hour=0
-----------------
2023 - 1 - 8 : 0
epoch.year=2023, epoch.month=1 (JA), epoch.day=5, epoch.hour=24
-----------------
2023 - 1 - 9 : 0
epoch.year=2023, epoch.month=1 (JA), epoch.day=6, epoch.hour=12
-----------------
2023 - 1 - 10 : 0
epoch.year=2023, epoch.month=1 (JA), epoch.day=7, epoch.hour=0
-----------------
2023 - 1 - 11

In [230]:
get_conversion_definition(2023,
                          DAYS_IN_24_HR_MONTH,
                          DAYS_IN_24_HR_MONTH_LEAP_YR,
                          36)

[-1,
 Month(num_days=21, hours_in_days=[-1, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 24], month_abrv='JA'),
 Month(num_days=19, hours_in_days=[-1, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 24], month_abrv='FE'),
 Month(num_days=21, hours_in_days=[-1, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 24], month_abrv='MR'),
 Month(num_days=20, hours_in_days=[-1, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36], month_abrv='AR'),
 Month(num_days=21, hours_in_days=[-1, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 24], month_abrv='MA'),
 Month(num_days=20, hours_in_days=[-1, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36], month_abrv='JN'),
 Month(num_days=21, hours_in_days=[-1, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 24], month_abrv='JL'),
 Month(num_days=2